In [1]:
# Imports
import pandas as pd
import numpy as np

import plotly.express as px

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

C:\Users\katon\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


### Read and Clean Data

In [2]:
# Data read
path = "C:/Users/katon/Documents/JHU/DataVisualization/final_project/data/"

housing = pd.read_excel(path+"state_housing_inventory_10-21.xlsx")
population = pd.read_excel(path+"state_population_10-22.xlsx")
hpi = pd.read_excel(path+'state_HPI.xlsx')

population_proj = pd.read_excel(path+'population_projections.xls')

# Read into dictionary
homeless_dict = pd.read_excel(path+"state_homeless_counts_07-22.xlsx", sheet_name=None)


In [3]:
# clean housing
housing.set_index('State', inplace=True)

# clean population
population.set_index('State', inplace=True)

# clean population_proj
population_proj.set_index('State', inplace=True)

# clean hpi
hpi_new = pd.DataFrame(index=hpi.Abbreviation.unique(), columns=hpi.Year.unique())
for index in hpi.index:
    hpi_new.loc[hpi.loc[index, 'Abbreviation'], hpi.loc[index, 'Year']] = hpi.loc[index, 'HPI']
hpi = hpi_new

In [4]:
# Get est yearly pop change
diff_20_30 = (population_proj[2030] - population_proj[2020]) / 10
diff_30_40 = (population_proj[2040] - population_proj[2030]) / 10

# project based on differences
for year in range(2023, 2041):
    if year <= 2030:
        population[year] = population[year-1] + diff_20_30
    else:
        population[year] = population[year-1] + diff_30_40

In [5]:
# Clean homeless
for key in homeless_dict.keys():
    homeless_dict[key].columns = [col.replace(key, "") for col in homeless_dict[key].columns]
    homeless_dict[key].columns = [col.replace(",", "") for col in homeless_dict[key].columns]
    homeless_dict[key].columns = [col.replace(" ", "") for col in homeless_dict[key].columns]
    homeless_dict[key] = homeless_dict[key][['State', 'OverallHomeless']]
    homeless_dict[key].columns = ['State', key]
    homeless_dict[key].set_index('State', inplace=True)
homeless = pd.concat([df for df in homeless_dict.values()], axis=1)
homeless.columns = [int(col) for col in homeless.columns]
homeless = homeless[homeless.columns[::-1]]
homeless = homeless.drop(['AS', 'MP'])
homeless = homeless.astype(int)
homeless[2021] = ((homeless[2020] + homeless[2022])/2).astype(int)

    housing - 2010-2021
    population - 2010-2022
    homeless - 2022-2007
    hpi - 1975-2022

# check dfs before output

In [6]:
population

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040
State,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
US,309321666,311556874,313830990,315993715,318301008,320635163,322941311,324985539,326687501,328239523,331511512,332031554,333287557,3.358324e+08,3.383772e+08,3.409220e+08,3.434668e+08,3.460116e+08,3.485565e+08,3.511013e+08,3.536461e+08,3.557878e+08,3.579295e+08,3.600712e+08,3.622129e+08,3.643546e+08,3.664963e+08,3.686380e+08,3.707797e+08,3.729214e+08,3.750632e+08
AL,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185,5031362,5049846,5074296,5.086151e+06,5.098007e+06,5.109862e+06,5.121718e+06,5.133573e+06,5.145429e+06,5.157284e+06,5.169140e+06,5.171836e+06,5.174533e+06,5.177229e+06,5.179925e+06,5.182622e+06,5.185318e+06,5.188015e+06,5.190711e+06,5.193407e+06,5.196104e+06
AK,713910,722128,730443,737068,736283,737498,741456,739700,735139,731545,732923,734182,733583,7.376690e+05,7.417549e+05,7.458409e+05,7.499269e+05,7.540128e+05,7.580988e+05,7.621847e+05,7.662707e+05,7.690473e+05,7.718240e+05,7.746006e+05,7.773772e+05,7.801539e+05,7.829305e+05,7.857071e+05,7.884837e+05,7.912604e+05,7.940370e+05
AZ,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,7158024,7278717,7179943,7264877,7359197,7.456168e+06,7.553140e+06,7.650111e+06,7.747082e+06,7.844054e+06,7.941025e+06,8.037997e+06,8.134968e+06,8.227755e+06,8.320542e+06,8.413329e+06,8.506117e+06,8.598904e+06,8.691691e+06,8.784478e+06,8.877265e+06,8.970052e+06,9.062840e+06
AR,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,3009733,3017804,3014195,3028122,3045637,3.057368e+06,3.069098e+06,3.080829e+06,3.092560e+06,3.104290e+06,3.116021e+06,3.127751e+06,3.139482e+06,3.145656e+06,3.151829e+06,3.158003e+06,3.164177e+06,3.170351e+06,3.176524e+06,3.182698e+06,3.188872e+06,3.195045e+06,3.201219e+06
CA,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,39461588,39512223,39501653,39142991,39029342,3.936059e+07,3.969184e+07,4.002308e+07,4.035433e+07,4.068558e+07,4.101683e+07,4.134808e+07,4.167932e+07,4.195091e+07,4.222250e+07,4.249409e+07,4.276568e+07,4.303727e+07,4.330885e+07,4.358044e+07,4.385203e+07,4.412362e+07,4.439521e+07
CO,5047349,5121108,5192647,5269035,5350101,5450623,5539215,5611885,5691287,5758736,5784865,5811297,5839926,5.932288e+06,6.024651e+06,6.117013e+06,6.209375e+06,6.301738e+06,6.394100e+06,6.486462e+06,6.578825e+06,6.671417e+06,6.764010e+06,6.856602e+06,6.949194e+06,7.041787e+06,7.134379e+06,7.226972e+06,7.319564e+06,7.412156e+06,7.504749e+06
CT,3579114,3588283,3594547,3594841,3594524,3587122,3578141,3573297,3571520,3565287,3597362,3623355,3626205,3.626971e+06,3.627737e+06,3.628503e+06,3.629269e+06,3.630035e+06,3.630801e+06,3.631567e+06,3.632333e+06,3.626484e+06,3.620634e+06,3.614785e+06,3.608935e+06,3.603086e+06,3.597236e+06,3.591387e+06,3.585537e+06,3.579688e+06,3.573838e+06
DE,899593,907381,915179,923576,932487,941252,948921,956823,965479,973764,992114,1004807,1018396,1.027876e+06,1.037356e+06,1.046836e+06,1.056315e+06,1.065795e+06,1.075275e+06,1.084755e+06,1.094235e+06,1.102450e+06,1.110665e+06,1.118881e+06,1.127096e+06,1.135311e+06,1.143526e+06,1.151741e+06,1.159957e+06,1.168172e+06,1.176387e+06


In [7]:
housing

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
State,,,,,,,,,,,,
US,131825383,132312816,132834047,133538615,134388318,135285123,136286436,137366902,138516439,139684244,140805345,142153010
AL,2174761,2181026,2191746,2203508,2213610,2225606,2238800,2254615,2270705,2284847,2292732,2313642
AK,307181,308498,310002,311243,312396,313933,315361,316910,318307,319854,326598,327890
AZ,2849425,2859978,2872651,2891186,2914100,2936981,2966542,3000102,3037005,3075981,3092669,3138871
AR,1318072,1324638,1330927,1338640,1345392,1352639,1360401,1369522,1379970,1389129,1368241,1380728
CA,13688147,13728423,13771156,13827794,13906691,13985614,14075264,14172348,14275353,14366336,14415759,14512281
CO,2215012,2226631,2239177,2261232,2287950,2316814,2347589,2384092,2422816,2464164,2500838,2540822
CT,1488645,1491803,1494237,1498157,1502807,1507299,1512528,1517251,1521019,1524992,1531475,1536344
DE,406498,408937,411192,414557,418666,423137,427667,432753,438617,443781,450146,457954


In [8]:
homeless

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
State,,,,,,,,,,,,,,,,
AK,1642,1646,1992,1863,2128,1913,1946,1784,1956,1940,1845,2016,1907,1949,2134,2320
AL,5452,5387,6080,6046,5558,5209,4689,4561,3970,4111,3793,3434,3261,3351,3551,3752
AR,3836,3255,2852,2762,3424,4214,3812,2936,2560,2463,2467,2712,2717,2366,2412,2459
AZ,14646,12488,14721,13711,10504,11302,10562,10495,9896,9707,8947,9865,10007,10979,12266,13553
CA,138986,136531,123678,123480,125128,120098,118552,113952,115738,118142,131532,129972,151278,161548,166534,171521
CO,14225,14747,15268,15482,15116,16768,9754,10028,9953,10550,10940,10857,9619,9846,10121,10397
CT,4482,4627,4605,4316,4456,4209,4448,4450,4047,3902,3388,3976,3033,2905,2917,2930
DC,5320,6044,6228,6539,6546,6954,6865,7748,7298,8350,7473,6904,6521,6380,5395,4410
DE,1061,933,1130,982,1035,1008,946,901,953,1070,994,1082,921,1165,1767,2369


In [9]:
hpi

,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
AL,100.0,106.68,117.39,127.17,140.77,147.52,153.99,153.8,161.56,170.13,174.78,183.05,189.67,194.35,197.63,201.01,205.81,211.54,218.56,225.89,235.37,243.2,252.31,261.89,268.45,276.96,288.61,294.88,301.72,311.55,332.28,358.03,372.71,375.81,367.3,351.54,339.65,334.6,332.68,337.31,344.96,354.16,365.24,381.48,398.62,414.49,457.4,528.53
AK,100.0,109.12,118.46,130.35,144.84,136.57,165.55,206.85,219.06,234.39,221.62,213.3,180.53,191.94,172.58,174.18,196.69,204.06,207.17,215.68,226.05,234.99,241.86,247.94,257.1,262.65,275.95,288.56,298.15,328.44,369.9,406.79,420.33,419.23,414.03,412.19,414.24,415.01,425.95,436.57,450.47,458.83,464.88,471.38,481.54,492.85,525.71,588.78
AZ,100.0,104.09,112.43,129.56,158.3,176.74,190.96,181.91,200.55,209.34,216.55,228.36,230.75,228.66,227.47,226.65,231.56,236.7,241.49,252.75,269.5,280.65,292.39,305.57,318.09,340.04,361.02,379.46,394.96,440.31,572.83,674.53,670.48,589.77,480.64,417.85,375.46,391.66,437.82,481.97,508.89,544.91,585.21,630.4,674.06,719.59,851.27,1024.78
AR,100.0,109.87,122.3,132.82,152.68,163.68,171.17,171.2,185.68,190.82,193.61,200.01,201.32,199.38,199.74,201.54,205.35,210.46,218.79,229.71,241.98,248.97,255.84,263.5,269.31,277.07,289.85,297.66,305.54,322.38,345.17,365.39,374.28,372.67,365.33,355.0,349.05,348.72,352.89,358.16,366.35,375.29,388.52,404.7,418.04,429.45,475.68,550.95
CA,100.0,114.18,138.8,163.12,189.09,219.25,241.44,234.01,247.53,254.83,268.74,285.01,308.83,359.5,438.95,467.7,465.93,458.31,435.17,421.73,411.09,410.75,422.09,456.32,487.22,554.91,620.3,685.15,750.59,896.57,1118.42,1234.32,1185.77,956.6,812.77,786.89,742.41,741.63,818.75,931.64,985.67,1062.85,1131.14,1203.68,1244.06,1275.05,1435.39,1660.19
CO,100.0,108.72,123.73,145.51,170.25,186.1,199.16,222.71,227.11,227.51,228.26,231.82,229.36,224.33,226.34,231.59,239.34,252.9,274.59,301.85,329.16,344.85,363.42,380.84,409.04,462.5,500.23,521.39,526.14,541.85,568.68,587.88,592.6,586.14,576.68,553.15,535.87,539.95,568.67,620.49,679.16,749.32,814.45,881.04,922.98,949.56,1071.01,1244.62
CT,100.0,106.23,111.76,131.37,156.82,167.24,180.75,187.54,201.53,227.21,257.73,304.65,369.13,411.47,406.77,386.64,365.16,359.14,349.11,342.02,339.42,343.28,347.06,361.84,379.47,414.58,446.8,484.63,514.42,571.38,642.1,679.42,680.1,655.68,608.89,584.43,563.55,546.92,542.78,542.73,550.19,550.83,557.3,566.3,578.71,596.22,666.34,756.36
DE,100.0,104.46,101.33,108.64,115.35,133.65,145.01,148.51,155.46,169.62,182.12,199.92,222.17,249.61,273.62,280.72,284.48,285.76,284.91,281.44,281.61,286.41,290.29,297.91,308.29,327.68,350.5,374.93,399.4,450.79,522.12,568.22,583.43,566.25,528.97,504.21,476.31,462.46,466.23,477.39,488.52,498.08,508.61,528.62,546.08,564.34,624.62,709.86
DC,100.0,110.41,124.34,143.73,169.59,181.19,178.32,168.2,183.33,195.86,206.43,223.35,248.43,293.53,329.61,341.84,343.05,350.31,345.12,342.84,337.91,345.65,345.83,362.34,389.23,445.38,508.37,573.55,628.02,731.33,898.08,1001.99,1029.87,979.15,928.12,929.21,938.31,967.56,1032.8,1125.07,1191.11,1270.76,1334.41,1398.28,1451.7,1482.84,1584.75,1711.91
FL,100.0,98.11,102.38,113.29,128.05,143.24,151.15,150.86,170.92,175.16,177.24,183.8,188.4,196.51,204.51,207.9,211.03,215.85,219.94,221.96,227.97,233.73,240.63,250.65,259.81,279.84,306.65,334.55,362.27,418.58,522.79,605.01,592.02,502.08,413.37,373.53,347.86,344.72,370.22,408.91,444.89,485.63,526.2,568.1,599.69,630.99,726.41,890.35


In [10]:
# Output
population.to_excel(path+"final_data/population.xlsx")
housing.to_excel(path+"final_data/housing.xlsx")
homeless.to_excel(path+"final_data/homeless.xlsx")
hpi.to_excel(path+"final_data/hpi.xlsx")

<ipython-input-10-e53bd0eca391>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  population.to_excel(path+"final_data/population.xlsx")
<ipython-input-10-e53bd0eca391>:3: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  housing.to_excel(path+"final_data/housing.xlsx")
<ipython-input-10-e53bd0eca391>:4: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  homeless.to_excel(path+"final_data/homeless.xlsx")
<ipython-input-10-e53bd0eca391>:5: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  hpi.to_excel(path+"final_data/hpi.xlsx")
